In [99]:
import sys
sys.path.insert(0, "./consul")
import consul as cs
import importlib
importlib.reload(cs)
import pandas as pd

# TODO:
#  - produce some basic stats

# Load external inventory from CSV
inventory = pd.read_csv('./inventory.csv')
consul_url = 'http://localhost:8500'
inventory_endpoint = 'v1/kv/inventory/bos%2B'

# Update consul inventory (delete all current values, add new values)
# This is optional obviously if you have the correct inventory present
# df = pandas dataframe with inventory data
# action = delete or set
# consul_url, inventory_endpoint - correct urls
# keys = array of keys that are supposed to be used; if empty, all attrs in csv will be used
# mac_addr_col = name of column with mac address
cs.modify_inventory(inventory, 'delete', consul_url, None, 'mac')
cs.modify_inventory(inventory, 'set', consul_url, None, 'mac')

# Load inventory from consul
df_inventory = cs.get_inventory(consul_url, inventory_endpoint)
df_inventory

# Write loaded invnetory back (delete it before, so that we can check)
cs.modify_inventory(inventory, 'delete', consul_url, None, 'mac')

cs.modify_inventory(inventory, 'delete', consul_url, None, 'mac')
cs.modify_inventory(inventory[inventory.building.notnull()], 'set', consul_url, 
                    ['cooling_type','building','dim1','dim2','dim3','dim4','dim5','dim6','dim7','dim8','config_path','mac'], 'mac')

cs.get_inventory(consul_url, inventory_endpoint)

# Here is how to delete all inventory data (it will not delete folders, but those should not be there at all)
cs.modify_inventory(verb='delete_all', consul_url=consul_url, inventory_endpoint = inventory_endpoint)

cs.modify_inventory(inventory, 'set', consul_url, None, 'mac')
df_inventory = cs.get_inventory(consul_url, inventory_endpoint)
df_inventory

Dur (keys management): 0.02033376693725586
Dur (keys management): 0.017067670822143555
Dur (get inventory): 0.02143716812133789
Dur (keys management): 0.014702081680297852
Dur (keys management): 0.012921333312988281
Dur (keys management): 0.017739057540893555
Dur (get inventory): 0.018114089965820312
Dur (keys management): 0.012229681015014648
Dur (keys management): 0.022935152053833008
Dur (get inventory): 0.02788519859313965


,cooling_type,building,dim1,dim2,dim3,dim4,dim5,dim6,dim7,dim8,config_path,mac
0,air,a,abc,blah,ccc,ddd,eee,fff,ggg,hhh,template,00:01:aa:ff:ff:ff
1,air,a,cde,bleh,iii,jjj,kkk,lll,mmm,nnn,NaN,00:02:aa:ff:ff:ff
2,imm,b,efg,blih,ttt,sss,rrr,qqq,ppp,ooo,NaN,00:03:aa:ff:ff:ff
3,imm,b,abc,bloh,uuu,vv,www,xxx,yyy,zzz,NaN,00:04:aa:ff:ff:ff
4,air,a,cde,bluh,ccc,ddd,eee,fff,ggg,hhh,NaN,00:05:aa:ff:ff:ff


In [93]:
# Work in progress here ...

# Load instances of bos+ service, 2nd argument is consul node name
#df_instances = cs.get_instances(consul_url, 'miner')
cs.get_instances(consul_url, 'miner')
# Get the data together and print
#df_out = cs.merge_inventory_instances(df_inventory, df_instances)

#import dtale
#dtale.show(df_out)

#from IPython.display import HTML
#HTML(df_out[[x for x in df_out.columns if  x not in ['config_url', 'status_url', 'template_url','inv','bos']]].to_html(render_links=True, escape=False))

#import pandasql
#from pandasql import sqldf, load_meat, load_births
#pysqldf = lambda q: sqldf(q, globals())

#pysqldf("select template_url, count(*) as instances from df_out group by template_url;")
#df_out

,config_url,status_url,template_url,address,service_id,server_version,service_address,status
0,http://localhost:8500/ui/miner/kv/running/bos%...,http://localhost:8500/ui/miner/kv/running/bos%...,http://localhost:8500/ui/miner/kv/conf/bos+/An...,172.17.0.2,00:0a:35:ff:ff:ff,2022-09-13-0-11012d53-22.08-plus,192.168.0.2,
1,http://localhost:8500/ui/miner/kv/running/bos%...,http://localhost:8500/ui/miner/kv/running/bos%...,http://localhost:8500/ui/miner/kv/conf/bos+/An...,172.17.0.2,01:0a:35:ff:ff:ff,2022-09-13-0-11012d53-22.08-plus,192.168.0.3,
2,http://localhost:8500/ui/miner/kv/running/bos%...,http://localhost:8500/ui/miner/kv/running/bos%...,http://localhost:8500/ui/miner/kv/conf/bos+/An...,172.17.0.2,02:0a:35:ff:ff:ff,2022-09-13-0-11012d53-22.08-plus,192.168.0.4,


In [100]:
# Work in progress here ...

import time
import json
import base64
import pandas
import requests

running_endpoint = 'v1/kv/running/bos%2B'

def get_running(consul_url, running_endpoint):
    start_time = time.time()
    
    # Get list of all keys
    keys = requests.get('/'.join([consul_url,running_endpoint,'?keys'])).json()
    operations = [{"KV":{"Verb":"get","Key":key}} for key in keys if key.endswith('status')]
    
    # Helper function
    def is_json(myjson):
      try:
        json.loads(myjson)
      except ValueError as e:
        return False
      return True    

    # Read keys in batch (64 is the max value for consul transactions)
    results = []; i=0
    while i < len(operations):
        batch_res = [result["KV"] for result in requests.put('/'.join([consul_url,'v1/txn']), data = json.dumps(operations[i:i+64])).json()["Results"]]
        results = results + batch_res
        i = i+64
    
    # Decode base64 content stored in Value attribute
    [res.update({'Value':base64.b64decode(res['Value']).decode('UTF-8')}) for res in results]
    # Replace non-json values with invalid status
    [res.update({'Value':'{"parser":{"status":"invalid"}}'}) for res in results if not is_json(res['Value'])]

    df = pd.DataFrame(results)
    
    # Helper function to rearrange JSON into flat structure.
    def process(x):
        data = {}
        if 'bosminer_config_status' in x:
            data['la_ts_unix'] = x['bosminer_config_status']['last_applied']['timestamp_unix']
            data['la_ts_str'] = x['bosminer_config_status']['last_applied']['timestamp_str']
        x
        return data
    
    # Expand JSON in Value to columns and join in back
    df = df.join(df.Value.apply(lambda x: pd.Series(process(json.loads(x)),dtype="object")))
    return df

get_running(consul_url, running_endpoint)

,LockIndex,Key,Flags,Value,CreateIndex,ModifyIndex,la_ts_unix,la_ts_str
0,0,running/bos+/aaaa/bbb/00:0a:35:ff:ff:ff/status,0,"{\n ""bosminer_config_status"": {\n ""last_ap...",7256,7735,1665231470,2022-10-08 12:17:50.698107236 UTC
1,0,running/bos+/aaaa/bbb/01:0a:35:ff:ff:ff/status,0,"{\n ""bosminer_config_status"": {\n ""last_ap...",7738,8329,1665231470,2022-10-08 12:17:50.698107236 UTC
2,0,running/bos+/aaaa/bbb/02:0a:35:ff:ff:ff/status,0,"{\n ""bosminer_config_status"": {\n ""last_ap...",7741,7741,1665231470,2022-10-08 12:17:50.698107236 UTC
